In [1]:
import random
from sklearn.model_selection import StratifiedKFold
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
import os
!pip install torchinfo
from torchinfo import summary
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from sklearn.metrics import recall_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet18
import uuid
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output
!pip install git+https://github.com/openai/CLIP.git
from albumentations.core.composition import Compose, OneOf
from albumentations.pytorch import ToTensorV2
import albumentations as A    
    
import clip
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"


import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import wandb
 

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-4cyuv7zl
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-4cyuv7zl
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 150.0 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369409 sha256=c4cbf0632b4fa5f50e841f1c8e2080b4530d362bba801dddeb81aed502a4ece4
  Stored in directory: /tmp/pip-ephem-wheel-cache-9a4t57p4/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


## Настрока блокнота

In [2]:
class CFG:
    batch_size=80
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_epochs = 10
    wandb_project = "construction_equipment"
    wandb_entity = "vladsmirn_"
    img_size = 224
    n_fold = 5
    k_fold = 1
    seed=44
    
    id_notebook=str(uuid.uuid4())
    
print("id_notebook: " + CFG.id_notebook)
 

id_notebook: e84301f7-0ccb-4b77-aa71-1d708addd3d5


In [3]:
def get_transform(phase: str, img_size: int):
    if phase == 'train':
        return Compose([
#             A.Resize(height=img_size, width=img_size),
            A.RandomResizedCrop(height=img_size, width=img_size, scale=(0.97, 1.0)),
            A.Flip(p=0.5),
            A.RandomRotate90(p=0.1),
#             A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(p=0.1),
            A.OneOf([
                A.RandomBrightnessContrast(p=0.1),
                A.RandomGamma(p=0.1),
            ], p=0.1),
            A.OneOf([
                A.Blur(p=0.1),
                A.GaussianBlur(p=0.1),
                A.MotionBlur(p=0.1),
            ], p=0.1),
            A.OneOf([
                A.GaussNoise(p=0.1),
                A.ISONoise(p=0.1),
            ], p=0.2),
            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=img_size, width=img_size),
            A.Normalize(),
            ToTensorV2(),
        ])

In [4]:
DIR_TRAIN = "../input/construction-equipment/train/"
DIR_TEST = "../input/construction-equipment-test/test/"

PATH_TRAIN = "../input/construction-equipment/train.csv"
PATH_TEST = "../input/construction-equipment-test/test.csv"


##   Объявление модели


In [5]:
def run_wandb(wandb_api_key, cfg):
    wandb.require(experiment="service")
    wandb.login(key=wandb_api_key)
    _cfg = cfg() 
    cfg_wandb = dict((name, getattr(_cfg, name)) for name in dir(_cfg) if not name.startswith('__'))
    
    if hasattr(cfg, 'wandb_resume_id') and cfg.wandb_resume_id :
        run = wandb.init(project=cfg.wandb_project, 
                   entity=cfg.wandb_entity, 
                   id=cfg.wandb_resume_id, 
                   resume="allow", 
                   config=cfg_wandb)
    else: 
        run = wandb.init(project=cfg.wandb_project, 
                   entity=cfg.wandb_entity, 
                   settings=wandb.Settings(start_method="fork"), 
                   config=cfg_wandb,
                   resume="allow",
                   id=wandb.util.generate_id())
        
    model_logger = WandbLogger(project=cfg.wandb_project, log_model='all')
    
    return model_logger,run

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
# model_logger, _ = run_wandb(user_secrets.get_secret("WANDB_API_KEY"), CFG)

## Предсказание 

In [6]:
_, run = run_wandb(user_secrets.get_secret("WANDB_API_KEY"), CFG)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vladsmirn (vladsmirn_). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/loggers/wandb.py:353: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"


In [7]:
test_df = pd.read_csv(PATH_TEST)
test_df["path"] = test_df["ID_img"].apply(lambda image_name : f'{DIR_TEST}{image_name}')
test_df = test_df.drop(["class"], axis = 1)

class TestImageDataset(Dataset):
    def __init__(self, data_df, transform=None):
        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        image_path = self.data_df.iloc[idx]['path']
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image=image)['image'] 
 
        
        return image
    
    def __len__(self):
        return len(self.data_df)
    
# test_dataset = TestImageDataset(test_df,get_transform("train",CFG.img_size))

In [8]:
# test_dataset = TestImageDataset(test_df,get_transform("train",CFG.img_size))
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                            batch_size=1)

In [9]:
def run_model(loader,model):
    model.eval()
    predicts = []
    model.to(CFG.device)
    for imgs in tqdm(loader):   
        imgs = imgs.to(CFG.device)
        pred = model(imgs)
        prob = Categorical(logits=pred).probs.cpu().detach().tolist();
        predicts.extend(prob)
    return predicts

def get_model():
    model = models.convnext.convnext_small(pretrained=True)
    model.classifier[2] = nn.Linear(768, 8) 
    model.to(CFG.device)
    return model

def download_model_wandb(path_artifact,model,run):
    artifact = run.use_artifact(path_artifact, type='model')
    artifact_dir = artifact.download()
    checkpoint = os.path.join(artifact_dir, "model.ckpt")
    from collections import OrderedDict
    state_dict = torch.load(checkpoint,map_location ='cpu')['state_dict']
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[6:]  
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    return model 

In [10]:
from torch.distributions import Categorical

In [11]:
artifacts = ["vladsmirn_/construction_equipment/model-3b6ghnoz:v9", #mild-bee-60
             "vladsmirn_/construction_equipment/model-18u6rsng:v8", #eerie-vampire-94
             "vladsmirn_/construction_equipment/model-2ut170pm:v10",#paranormal-phantom-96
#              "vladsmirn_/construction_equipment/model-1k02nzr1:v7", #haunted-seance-93
             "vladsmirn_/construction_equipment/model-1if96ysv:v13",#valiant-durian-66
             
             "vladsmirn_/construction_equipment/model-3b6ghnoz:v9", #mild-bee-60
             "vladsmirn_/construction_equipment/model-18u6rsng:v8", #eerie-vampire-94
             "vladsmirn_/construction_equipment/model-2ut170pm:v10",#paranormal-phantom-96
 
             "vladsmirn_/construction_equipment/model-1if96ysv:v13",#valiant-durian-66
 
             "vladsmirn_/construction_equipment/model-3b6ghnoz:v9", #eerie-vampire-94
             "vladsmirn_/construction_equipment/model-3b6ghnoz:v9", #eerie-vampire-94
             "vladsmirn_/construction_equipment/model-3b6ghnoz:v9", #eerie-vampire-94
 
            ]

model = get_model()
first_run = True
seed = 1 
for artifact in artifacts:
    
    random.seed(seed)
    seed+=1
    
    test_dataset = TestImageDataset(test_df,get_transform("train",CFG.img_size))
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=1)
    
    model = download_model_wandb(artifact,model,run)
    if first_run == True:
        predict = np.array(run_model(test_loader,model))
        first_run = False
    else:
        predict += np.array(run_model(test_loader,model))
predict /= len(artifacts)
predict = np.argmax(predict,axis=1)
predict = predict.tolist()


Downloading: "https://download.pytorch.org/models/convnext_small-0c510722.pth" to /root/.cache/torch/hub/checkpoints/convnext_small-0c510722.pth


  0%|          | 0.00/192M [00:00<?, ?B/s]

wandb: Downloading large artifact model-3b6ghnoz:v9, 188.80MB. 1 files... Done. 0:0:12.0
100%|██████████| 2138/2138 [01:11<00:00, 30.10it/s]
wandb: Downloading large artifact model-18u6rsng:v8, 188.80MB. 1 files... Done. 0:0:10.3
100%|██████████| 2138/2138 [00:59<00:00, 36.08it/s]
wandb: Downloading large artifact model-2ut170pm:v10, 188.80MB. 1 files... Done. 0:0:11.4
100%|██████████| 2138/2138 [01:00<00:00, 35.61it/s]
wandb: Downloading large artifact model-1if96ysv:v13, 188.80MB. 1 files... Done. 0:0:11.3
100%|██████████| 2138/2138 [01:00<00:00, 35.59it/s]
wandb: Downloading large artifact model-3b6ghnoz:v9, 188.80MB. 1 files... Done. 0:0:0.1
100%|██████████| 2138/2138 [00:58<00:00, 36.36it/s]
wandb: Downloading large artifact model-18u6rsng:v8, 188.80MB. 1 files... Done. 0:0:0.1
100%|██████████| 2138/2138 [00:58<00:00, 36.35it/s]
wandb: Downloading large artifact model-2ut170pm:v10, 188.80MB. 1 files... Done. 0:0:0.1
100%|██████████| 2138/2138 [00:59<00:00, 35.94it/s]
wandb: Downlo

In [12]:
test_df["class"] = predict
test_df = test_df.drop(["path"], axis = 1)
test_df.head()

test_df.to_csv("submit.csv", index=False)